# Quadruped Forward/Inverse Kinematics
---
Kevin Walchko
Created: 24 Oct 2016

# Denavit-Hartenberg (DH)

Development of the DH forward kinematics for a quadruped robot. We will use the DH method to develop the symbolic equations and use python to simplify them.

![](https://github.com/walchko/pyGeckoRobots/blob/master/quadruped/robotis/docs/spider.png?raw=true)



In [25]:
from __future__ import print_function
from __future__ import division
import numpy as np
from sympy import symbols, sin, cos, pi, simplify
from math import radians as d2r
from math import degrees as r2d
from math import atan2, sqrt, acos, fabs

In [26]:
t1, t2, t3 = symbols('t1 t2 t3')
Lc, Lf, Lt = symbols('Lc Lf Lt')

The following class follows the traditional [DH convention](https://en.wikipedia.org/wiki/Denavit%E2%80%93Hartenberg_parameters). Where:

| Parameter | Definition|
|-----------|-----------|
| $\alpha_i$   | angle about common normal, from old z axis to new z axis (twist)|
| $d_i$        | offset along previous z to the common normal |
| $\theta_i$   | angle about previous z, from old x to new x |
| $a_i$        | length of the common normal. Assuming a revolute joint, this is the radius about previous z. |

In [27]:
class DH(object):
    def __init__(self):
        pass

    def fk(self, params):
        t = np.eye(4)
        for p in params:
            t = t.dot(self.makeT(*p))
        return t

    def makeT(self, a, alpha, d, theta):
        return np.array([  # classic DH
            [cos(theta), -sin(theta) * cos(alpha),  sin(theta) * sin(alpha), cos(theta) * a],
            [sin(theta),  cos(theta) * cos(alpha), -cos(theta) * sin(alpha), sin(theta) * a],
            [         0,               sin(alpha),               cos(alpha),              d],
            [         0,                        0,                        0,              1]
        ])

In [28]:
def eval(f, inputs):
    h = []
    for i in range(0, 3):
        tmp = (f[i,3]).subs(inputs)
        h.append(tmp.evalf())
    return h

The parameters are:

| i |$a_i$        | $\alpha_i$   | $d_i$   | $theta_i$  |
|---|-------------|--------------|---------|------------|
| 1 | $L_{coxa}$  | 90           | 0       | $\theta_1$ |
| 2 | $L_{femur}$ | 0            | 0       | $\theta_2$ |
| 3 | $L_{tibia}$ | 0            | 0       | $\theta_3$ |


In [5]:
# a, alpha, d, theta
params = [
    [Lc, pi/2, 0, t1],
    [Lf,    0, 0, t2],
    [Lt,    0, 0, t3]
]
dh = DH()
t = dh.fk(params)
t = eval(t,[])

In [29]:
for i in t:
    print(simplify(i))

1.0*(Lc + Lf*cos(t2) + Lt*cos(t2 + t3))*cos(t1)
1.0*(Lc + Lf*cos(t2) + Lt*cos(t2 + t3))*sin(t1)
1.0*Lf*sin(t2) + 1.0*Lt*sin(t2 + t3)


In [7]:
def fk(t1, t2, t3, Lc, Lf, Lt):
    t1 = d2r(t1)
    t2 = d2r(t2)
    t3 = d2r(t3)
    return np.array([
        (Lc + Lf*cos(t2) + Lt*cos(t2 + t3))*cos(t1),
        (Lc + Lf*cos(t2) + Lt*cos(t2 + t3))*sin(t1),
        Lf*sin(t2) + 1.0*Lt*sin(t2 + t3)
    ])

# Inverse Kinematics

Often, we need to find the angles of the leg joints given a location (x,y,z) of the foot. To calculate this, remember the leg is composed of 3 revolute joints. The shoulder is in one plane and both the femur joint and tibia joint are in another plane.

**insert picture here**

## Useful Trigonometry

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/49/Triangle_with_notations_2.svg/200px-Triangle_with_notations_2.svg.png)

The **law of cosines** for calculating one side of a triangle when the angle
opposite and the other two sides are known. Can be used in conjunction with
the law of sines to find all sides and angles.

## Derivation

So let's start with the shoulder, look down from the top, we see the x-y plane. We can calculate the it's rotation $\theta_1$ by using x and y of the foot location.

$$\theta_1 = atan2(y,x)$$

Next, let's look at the plane which the next 2 joints lie in. Now we will calculate the $\theta_2$ and $\theta_3$ angles.

$$ m = \sqrt{x^2 + y^2} $$

$$ f = m - L_{coxa} $$

$$ \beta_1 = atan2(f,z) $$

$$ d = \sqrt{f^2 + z^2} $$

$$ \beta_2 = \arccos( \frac{L_{femur}^2+d^2-L_{tibia}^2}{2 d L_{femur}} ) $$

$$ \theta_2 = \beta_1 + \beta_2 $$

$$\theta_3 = \arccos( \frac{L_{femur}^2+L_{tibia}^2-d^2}{2 L_{femur} L_{tibia}} ) $$

Now finally, you should notice the definition is different between forward and reverse kinematics. So let's fix that ...


$$ \theta_{2fk} = \theta_{2ik} - \pi $$
$$ \theta_{3fk} = \theta_{3ik} - \pi/2 $$


In [13]:
def ik(x, y, z, Lc, Lf, Lt):
    t1 = atan2(y, x)
    f = sqrt(x**2 + y**2) - Lc
    
    # depending on z, you have to do a different atan
    if z < 0.0:
        b1 = atan2(f, fabs(z))
    else:
        b1 = atan2(z, f)
    d = sqrt(f**2 + z**2)  
    b2 = acos((Lf**2 + d**2 - Lt**2) / (2.0 * Lf * d))
    t2 = b1 + b2
    t3 = acos((Lf**2 + Lt**2 - d**2) / (2.0 * Lf * Lt))

    t3 -= pi  # fix to align fk and ik frames
    if z < 0.0:
        t3 -= pi/2

    return [r2d(t1), r2d(t2),r2d(t3)]

In [16]:
pts = fk(0,45,-60, 10, 40, 100)
# print('pts: {} {} {}'.format(*pts))

angles = ik(pts[0], pts[1], pts[2], 10, 40, 100)
print('angles: {:.1f} {:.1f} {:.1f}'.format(*angles))

angles: 0.0 45.0 -60.0


In [24]:
# from random import uniform
# for i in range(10):
#     a=uniform(-90,90)
#     b=uniform(0,90)
#     c=uniform(-180,0)
#     pts = fk(a,b,c, 10, 40, 100)
#     angles = ik(pts[0], pts[1], pts[2], 10, 40, 100)
#     if angles:
#         if a-angles[0]>0.1 or b-angles[1]>0.1 or c-angles[2]>0.1:
#             print('in: {:.1f} {:.1f} {:.1f} out: {:.1f} {:.1f} {:.1f}'.format(a,b,c, *angles))

In [20]:
# def printError(pts, pts2, angles, angles2):
#     print('****************************************************')
#     print('angles (orig):', angles)
#     print('angles2 from ik(pts): {:.2f} {:.2f} {:.2f}'.format(*angles2))
#     print('pts from fk(orig): {:.2f} {:.2f} {:.2f}'.format(*pts))
#     print('pts2 from fk(angle2): {:.2f} {:.2f} {:.2f}'.format(*pts2))
#     print('diff [deg]: {:.2f}'.format(np.linalg.norm(np.array(angles) - np.array(angles2))))
#     print('diff [mm]: {:.2f}'.format(np.linalg.norm(pts - pts2)))
#     print('\nExiting\n')
#     print('****************************************************')

# def fk_ik():
#     cox = 10
#     fem = 40
#     tib = 100
#     for a in range(-45, 45, 5):
#         for b in range(0,90,5):
#             for g in range(-90,0,5):
#                 print('------------------------------------------------')
#                 a1 = [a,b,g]
#                 pts = fk(a1[0],a1[1],a1[2], cox, fem, tib)
#                 a2 = ik(pts[0], pts[1], pts[2], cox, fem, tib)
#                 pts2 = fk(a2[0], a2[1], a2[2], cox, fem, tib)

#                 print('points:', pts)
# #                 print(type(pts))
# #                 print(pts - pts2)
# #                 print(np.linalg.norm(pts - pts2))
                
# #                 print(a1)
# #                 print(type(a1))
# #                 print(a2)
# #                 print(type(a2))

#                 angle_error = np.linalg.norm(np.array(a1) - np.array(a2))
#                 pos_error = np.linalg.norm(pts - pts2)
#                 # print(angle_error, pos_error)

#                 if angle_error > 0.0001:
#                     print('Angle Error')
#                     printError(pts, pts2, angles, angles2)
#                     exit()

#                 elif pos_error > 0.0001:
#                     print('Position Error')
#                     printError(pts, pts2, angles, angles2)
#                     exit()

#                 else:
#                     print('Angle: {:.1f} {:.1f} {:.1f}'.format(angles[0], angles[1], angles[2]))
#                     print('Pos: {:.1f} {:.1f} {:.1f}'.format(pts[0], pts[1], pts[2]))
#                     print('Error(deg,mm): {:.2f} {:.2f}\n'.format(angle_error, pos_error))
        
# fk_ik()

In [ ]:
print(sqrt)


-----------

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.